In [1]:
# Install core and visualization libraries
!pip install --upgrade --quiet json-repair networkx langchain-core langchain-google-vertexai langchain-experimental langchain-community langchain_google_genai chromadb

# Install additional libraries for scraping and plotting
!pip install --upgrade --quiet requests beautifulsoup4 matplotlib ipywidgets gravis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.4 MB/s eta 0:0

In [2]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
import networkx as nx
import matplotlib.pyplot as plt
import gravis as gv

from IPython.display import display, clear_output
import ipywidgets as widgets
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
from langchain.indexes import GraphIndexCreator
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [5]:
# Define the content for the context file
context_content = """
# Context for LLM: Functions and Framework for Circular Seating Arrangement Problem

# Description:
# This framework is designed to solve circular seating arrangement problems involving six representatives.
# It provides functions for defining and validating conditions, generating valid arrangements, and querying
# specific seating-related questions.

# Instructions:
# 1. Use the `find_valid_arrangements` function to find all valid arrangements based on specified conditions.
# 2. Use `is_valid_arrangement` to validate specific arrangements.
# 3. For querying specific relationships (e.g., neighbors or someone sitting between two people), use the
#    query functions (`query_neighbors`, `query_between`).
# 4. Ensure that all required conditions are implemented as functions and passed to `find_valid_arrangements`.

import itertools

# Define the list of representatives
representatives = ['K', 'L', 'M', 'N', 'O', 'P']

# Function to check if two people sit immediately next to each other
def sits_next_to(arrangement, person1, person2):
    ""Checks if person1 and person2 are adjacent in the arrangement.""
    n = len(arrangement)
    return abs(arrangement.index(person1) - arrangement.index(person2)) == 1 or \
           abs(arrangement.index(person1) - arrangement.index(person2)) == n - 1

# Function to check if a person sits next to any of a list of people
def sits_next_to_any(arrangement, person, others):
    ""Checks if person sits next to any person in the 'others' list.""
    n = len(arrangement)
    return any(abs(arrangement.index(person) - arrangement.index(other)) == 1 or
               abs(arrangement.index(person) - arrangement.index(other)) == n - 1
               for other in others)

# Function to check if a person does not sit next to another person
def does_not_sit_next_to(arrangement, person1, person2):
    ""Checks if person1 does not sit next to person2.""
    n = len(arrangement)
    return abs(arrangement.index(person1) - arrangement.index(person2)) != 1 and \
           abs(arrangement.index(person1) - arrangement.index(person2)) != n - 1

# Function to check the condition where one person sits next to another, and then a third person does not sit next to the second
def sits_next_to_and_not_next_to(arrangement, person1, person2, person3):
    ""Ensures person1 sits next to person2, and person3 does not sit next to person2.""
    n = len(arrangement)
    pos1 = arrangement.index(person1)  # Position of person1
    pos2 = arrangement.index(person2)  # Position of person2
    pos3 = arrangement.index(person3)  # Position of person3

    if abs(pos1 - pos2) == 1 or abs(pos1 - pos2) == n - 1:
        return abs(pos2 - pos3) != 1 and abs(pos2 - pos3) != n - 1
    return True

# Helper function to normalize circular permutations
def normalize(arrangement):
    ""Returns the lexicographically smallest rotation of the arrangement.""
    n = len(arrangement)
    rotations = [arrangement[i:] + arrangement[:i] for i in range(n)]
    return min(rotations)

# Function to return all valid arrangements based on a list of condition functions
def find_valid_arrangements(arrangements, conditions):
    ""Generates all valid arrangements that satisfy the provided conditions.""
    unique_valid_arrangements = set()
    for arrangement in arrangements:
        if all(condition(arrangement) for condition in conditions):
            normalized_arrangement = normalize(arrangement)
            unique_valid_arrangements.add(tuple(normalized_arrangement))
    return unique_valid_arrangements

# Function to check if a given arrangement is valid based on the unique_valid_arrangements set
def is_valid_arrangement(arrangement, valid_set):
    ""Checks if a specific arrangement is valid.""
    normalized_arrangement = normalize(arrangement)
    return tuple(normalized_arrangement) in valid_set

# Function to return a dictionary mapping chair positions to representatives
def seating_positions_from_arrangement(arrangement):
    ""Maps chair positions (1 to n) to representatives.""
    return {i + 1: person for i, person in enumerate(arrangement)}

# Function to find who sits to the left of someone
def who_sits_on_left(seating_positions, person):
    ""Finds the person sitting to the left (counter-clockwise) of the given person.""
    n = len(seating_positions)
    pos = [key for key, value in seating_positions.items() if value == person][0]
    left_pos = pos - 1 if pos - 1 > 0 else n  # Wrap around to the last position
    return seating_positions[left_pos]

# Function to find who sits to the right of someone
def who_sits_on_right(seating_positions, person):
    ""Finds the person sitting to the right (clockwise) of the given person.""
    n = len(seating_positions)
    pos = [key for key, value in seating_positions.items() if value == person][0]
    right_pos = pos + 1 if pos + 1 <= n else 1  # Wrap around to the first position
    return seating_positions[right_pos]

# Function to correctly find the person sitting between two others
def who_sits_between(seating_positions, person1, person2):
    ""Finds the person sitting directly between person1 and person2.""
    pos1 = [key for key, value in seating_positions.items() if value == person1][0]
    pos2 = [key for key, value in seating_positions.items() if value == person2][0]
    n = len(seating_positions)
    if abs(pos1 - pos2) == 2 or abs(pos1 - pos2) == n - 2:
        between_pos = (pos1 + 1) if abs(pos1 - pos2) == 2 else (pos2 + 1)
        between_pos = between_pos if between_pos <= n else 1
        return seating_positions[between_pos]
    return None

# Function to query neighbors of a specific person in a given seating arrangement
def query_neighbors(seating_position, person):
    ""Queries the neighbors of a person in the seating arrangement.""
    seating_positions = seating_positions_from_arrangement(seating_position)
    left_person = who_sits_on_left(seating_positions, person)
    right_person = who_sits_on_right(seating_positions, person)
    print(f"{person} has {left_person} on the left and {right_person} on the right.")

# Function to query who sits between two specific people in a given arrangement
def query_between(seating_position, person1, person2):
    ""Queries who sits between two people in the seating arrangement.""
    seating_positions = seating_positions_from_arrangement(seating_position)
    person_between = who_sits_between(seating_positions, person1, person2)
    if person_between:
        print(f"{person_between} sits between {person1} and {person2}.")
    else:
        print(f"No one sits between {person1} and {person2}.")
"""


In [6]:
# Create the context file
context_file_path = "seating_arrangement_context.txt"

with open(context_file_path, "w") as file:
    file.write(context_content)

print(f"Context file created successfully at: {context_file_path}")

Context file created successfully at: seating_arrangement_context.txt


In [ ]:
os.environ["GOOGLE_API_KEY"] = "your_key"

In [ ]:
import os
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Set up the Google API Key
try:
    google_api_key = os.environ["GOOGLE_API_KEY"]
except KeyError:
    raise ValueError("Google API Key not found in environment variables. Please set the 'GOOGLE_API_KEY' environment variable.")

# Initialize the ChatGoogleGenerativeAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    verbose=True
)

# Read the context from the generated text file
try:
    with open("seating_arrangement_context.txt", "r") as file:
        context_content = file.read()
except FileNotFoundError:
    raise FileNotFoundError("Context file 'seating_arrangement_context.txt' not found.")

# Create a Prompt Template that includes the context
prompt_template = PromptTemplate(
    input_variables=["input"],
    template=f"""{context_content}

You are an intelligent assistant designed to process seating arrangement queries.
Using the above functions and framework as context:
- Write Python code to validate conditions and generate valid seating arrangements when given a problem statement.
- Write Python code to answer specific queries about the seating arrangement.

User Input: {{input}}
Response:"""
)

# Set up memory for the conversation chain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
print(memory)
print(type(memory))
memory.messages.append(context_content)
print(memory)
print(type(memory))
# # Create the conversation chain using the 'RunnableWithMessageHistory'
# conversation_chain = RunnableWithMessageHistory(
#     llm=llm,
#     memory=memory,
#     prompt=prompt_template,
#     runnable=llm,
#     get_session_history=lambda: memory.messages  # Accessing messages from memory
# )

# # Simulating user input (initial problem statement)
# initial_problem_statement = """Exactly six trade representatives negotiate a treaty: K, L, M, N, O, P.
# There are exactly six chairs evenly spaced around a circular table.
# The chairs are numbered 1 through 6, with successively numbered chairs next to each other and chair number 1 next to chair number 6.
# Each chair is occupied by exactly one of the representatives. The following conditions apply:
# - P sits immediately next to N.
# - L sits immediately next to M, N, or both.
# - K does not sit immediately next to M.
# - If O sits immediately next to P, O does not sit immediately next to M."""

# # Add initial problem statement as input to the conversation chain
# response_1 = conversation_chain.invoke(input=initial_problem_statement)
# print("Initial Problem Response:")
# print(response_1)

# # Querying the seating arrangement solution
# query_1 = "Can you provide a valid seating arrangement based on the above conditions?"
# response_2 = conversation_chain.invoke(input=query_1)
# print("\nSeating Arrangement Query Response:")
# print(response_2)

# # Querying another specific query
# query_2 = "Who sits next to K?"
# response_3 = conversation_chain.invoke(input=query_2)
# print("\nSeating Arrangement Query Response (Who sits next to K?):")
# print(response_3)

# # Access and print the conversation history
# conversation_history = memory.messages  # Corrected to access messages from memory
# print("\nConversation History:")
# for message in conversation_history:
#     print(f"{message['role'].capitalize()} Message: {message['content']}")

chat_memory=InMemoryChatMessageHistory(messages=[]) return_messages=True memory_key='chat_history'
<class 'langchain.memory.buffer.ConversationBufferMemory'>


AttributeError: 'ConversationBufferMemory' object has no attribute 'messages'

In [ ]:
import os
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv
from langchain.memory import ConversationSummaryMemory

# Load environment variables
load_dotenv()

# Initialize Gemini model (ChatGoogleGenerativeAI)
chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # You can use the appropriate Gemini model here
    temperature=0.2,
    verbose=True
)

# Set up memory (ConversationSummaryMemory)
memory = ConversationSummaryMemory(
    memory_key="messages",  # Key for storing message history
    return_messages=True,  # Ensures memory tracks messages, not just completions
    llm=chat  # LLM model to generate summaries
)

# Load context content from the 'seating_arrangement_context.txt' file
context_file_path = "seating_arrangement_context.txt"

try:
    with open(context_file_path, "r") as file:
        context_content = file.read()
except FileNotFoundError:
    raise FileNotFoundError(f"Context file '{context_file_path}' not found.")

# Preload the context as the system message into memory
# memory.chat_memory.add_message("system", context_content)  # Add context to memory as a system message

# Set up the ChatPromptTemplate to include the conversation history
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),  # Placeholder for conversation history
        HumanMessagePromptTemplate.from_template("{content}"),  # User's message as input
    ]
)

# Create the chain with LLM, prompt, and memory
chain = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=memory
)

content = context_content
# Querying or sending a statement
result = chain({"content": content})  # Passing the user input to the chain
print(result['text'])  # Displaying the response generated by Gemini

This code provides a robust framework for solving circular seating arrangement problems.  Here's how it works and how you can use it to answer specific queries:


**1. Generating Valid Arrangements:**

The `generate_valid_arrangements()` function is the core of the solution. It iterates through all possible permutations of the representatives, checks each arrangement against the four defined conditions, normalizes the arrangements to account for circularity, and returns a set of unique valid arrangements.

**2. Querying Arrangements:**

The functions `who_sits_on_left()`, `who_sits_on_right()`, and `who_sits_between()` allow you to query the seating arrangements.  They take a seating arrangement (as a dictionary from chair number to person) and return the requested information.  The helper functions `query_neighbors` and `query_between` simplify the process of using these query functions.

**Example Usage:**

```python
valid_arrangements = generate_valid_arrangements()
print(f"Number o

In [ ]:
problem_statement = """
Exactly six trade representatives negotiate a treaty: A, B, C, D, E, F. There are exactly six chairs evenly spaced around a circular table. The chairs are numbered 1 through 6, with successively numbered chairs next to each other and chair number 1 next to chair number 6. Each chair is occupied by exactly one of the representatives. The following conditions apply:

- D sits immediately next to F.
- B sits immediately next to C, D, or both.
- A does not sit immediately next to C.
- If E sits immediately next to A, E does not sit immediately next to C.

Give code to get all valid arrangements based on given problem
"""

result = chain({"content": problem_statement})  # Passing the user input to the chain
print(result['text'])  # Displaying the response generated by Gemini

Here's Python code to solve this circular seating arrangement problem, along with explanations and suggestions for improvement:

```python
import itertools

def is_adjacent(arrangement, person1, person2):
    """Checks if two people are adjacent in the arrangement."""
    index1 = arrangement.index(person1)
    index2 = arrangement.index(person2)
    return abs(index1 - index2) == 1 or (index1 == 0 and index2 == len(arrangement) - 1) or (index1 == len(arrangement) - 1 and index2 == 0)

def check_conditions(arrangement):
    """Checks if an arrangement satisfies all conditions."""
    #Note: Condition 4 has been corrected to reflect the intended meaning.  The original condition used 'O' which was undefined.  I've assumed it was meant to be 'E'.
    if not is_adjacent(arrangement, 'D', 'F'): return False
    if not (is_adjacent(arrangement, 'B', 'C') or is_adjacent(arrangement, 'B', 'D') or (is_adjacent(arrangement, 'B', 'C') and is_adjacent(arrangement, 'B', 'D'))): return False
    if 

In [ ]:
query = """

Exactly six trade representatives negotiate a treaty: A, B, C, D, E, F. There are exactly six chairs evenly spaced around a circular table. The chairs are numbered 1 through 6, with successively numbered chairs next to each other and chair number 1 next to chair number 6. Each chair is occupied by exactly one of the representatives. The following conditions apply:

- D sits immediately next to F.
- B sits immediately next to C, D, or both.
- A does not sit immediately next to C.
- If E sits immediately next to A, E does not sit immediately next to C.

Which one of the following seating arrangements of the six representatives in chairs 1 through 6 would NOT violate the stated conditions?
- A, B, D, C, E, F
- A, F, B, D, C, E
- A, F, B, E, D, C
- A, E, B, D, C, F
- A, D, F, B, E, C
"""

result = chain({"content": query})  # Passing the user input to the chain
print(result['text'])  # Displaying the response generated by Gemini

The fourth condition in your problem statement contains a typo; 'O' should be 'E'.  Let's correct that and create a Python solution.

Here's a refined Python solution incorporating error handling, user input, and suggestions for visualization:

```python
import itertools

def define_conditions():
    """Defines the seating conditions."""
    conditions = [
        lambda arrangement: are_adjacent(arrangement, 'D', 'F'),
        lambda arrangement: is_adjacent_to_any(arrangement, 'B', ['C', 'D']),
        lambda arrangement: not are_adjacent(arrangement, 'A', 'C'),
        lambda arrangement: not (are_adjacent(arrangement, 'E', 'A') and are_adjacent(arrangement, 'E', 'C'))
    ]
    return conditions

def are_adjacent(arrangement, rep1, rep2):
    """Checks if two representatives are adjacent in the arrangement."""
    index1 = arrangement.index(rep1)
    index2 = arrangement.index(rep2)
    return abs(index1 - index2) == 1 or (index1 == 0 and index2 == 5) or (index1 == 5 and index2 == 